In [167]:
# models 
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# clustering 
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering


import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

#Distance Calculation
from scipy.spatial import distance

In [168]:
model=load_model(r"D:\Ambis Kuliah\Semester 7\NAIST Internship\Breast Cancer Project\Program\bcancer_modelv3.h5")
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
#model.summary()

In [169]:
b = pd.read_excel(r'D:\Ambis Kuliah\Semester 7\NAIST Internship\Breast Cancer Project\Program\featuree.xlsx')


In [170]:
ward  = AgglomerativeClustering(n_clusters=9, metric='euclidean', linkage='ward')


labels = ward.fit_predict(b.values)

In [171]:
def extract_features(file, model):
    img = load_img(file, target_size=(224,224))
    img = np.array(img) 
    reshaped_img = img.reshape(1,224,224,3) 
    
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [211]:
lobular_carcinoma_in_situ = r"D:\Ambis Kuliah\Semester 7\NAIST Internship\Breast Cancer Project\Program\Validation Cluster\Preprocessing\Lobular Carcinoma In Situ.jpg"

invasive_lobular_carninoma = r"D:\Ambis Kuliah\Semester 7\NAIST Internship\Breast Cancer Project\Program\Validation Cluster\Preprocessing\Invasive Lobular Carcinoma.jpg"

In [212]:
in_situ_feat = {}

feat = extract_features(lobular_carcinoma_in_situ, model)
in_situ_feat[0] = feat

filenames = np.array(list(in_situ_feat.keys()))

insitu_feat = np.array(list(in_situ_feat.values()))
insitu_feat = insitu_feat.reshape(-1,512)

insitu_feat.shape

1/1 [==============================] - 0s 26ms/step


(1, 512)

In [213]:
in_vasive_feat = {}

feat = extract_features(invasive_lobular_carninoma, model)
in_vasive_feat[0] = feat

filenames = np.array(list(in_vasive_feat.keys()))

invasive_feat = np.array(list(in_vasive_feat.values()))
invasive_feat = invasive_feat.reshape(-1,512)

#print(invasive_feat)


1/1 [==============================] - 0s 24ms/step


In [215]:
#a = pd.DataFrame(insitu_feat)
#a.to_excel(r'D:\Ambis Kuliah\Semester 7\NAIST Internship\Breast Cancer Project\Program\insitu_feat.xlsx')

In [216]:
val_data = pd.read_excel(r'D:\Ambis Kuliah\Semester 7\NAIST Internship\Breast Cancer Project\Program\val_feature.xlsx')

In [219]:
val_feat = val_data.values
#print(val_feat[0])

In [220]:
cluster_centroids = np.empty((ward.n_clusters, b.shape[1]))
for cluster_label in range(ward.n_clusters):
    cluster_centroid = np.mean(b[ward.labels_ == cluster_label], axis=0)
    cluster_centroids[cluster_label] = cluster_centroid

In [221]:
#centroid = pd.DataFrame(cluster_centroids)
#centroid.to_excel(r"D:\Ambis Kuliah\Semester 7\NAIST Internship\Breast Cancer Project\Program\centroid_cluster.xlsx")

In [222]:
centroid_feat = pd.read_excel(r"D:\Ambis Kuliah\Semester 7\NAIST Internship\Breast Cancer Project\Program\centroid_cluster.xlsx")
centroid_feat = centroid_feat.values

#print(centroid_feat[8])

In [223]:
dist = np.eye(N=2,M=9,dtype=float)
#In Situ Distance
for i in range(0,2):
    for j in range(0,9):
        dist[i][j] = distance.euclidean(centroid_feat[j], val_feat[i])


In [225]:
dist_df = pd.DataFrame(dist)
dist_df.style.background_gradient(cmap='YlOrRd')

,0,1,2,3,4,5,6,7,8
0,4516.332914,18018.586169,4123.392438,10364.212207,2710.285480,7458.265913,10391.888067,27353.317939,15051.875495
1,7947.520615,29754.603396,15807.713968,1847.706776,11170.783536,5663.621585,22211.434335,38921.801850,3582.575858


In [226]:
from sklearn.metrics import pairwise

In [259]:
dist2 = np.eye(N=2,M=9,dtype=float)
#In Situ Distance
for i in range(0,2):
    for j in range(0,9):
        dist2[i][j] = distance.correlation(centroid_feat[j], val_feat[i])

In [260]:
dist_df2 = pd.DataFrame(dist2)
dist_df2.style.background_gradient(cmap='YlOrRd')

,0,1,2,3,4,5,6,7,8
0,0.002395,0.007204,0.000236,0.002277,0.002334,0.001937,0.002755,0.018630,0.002512
1,0.001087,0.020477,0.003616,0.000345,0.005813,0.003348,0.012025,0.035100,0.000358


In [261]:
dist2_norm = (dist2-np.min(dist2))/(np.max(dist2)-np.min(dist2))

In [262]:
dist_df2 = pd.DataFrame(dist2_norm)
dist_df2.style.background_gradient(cmap='YlOrRd')

,0,1,2,3,4,5,6,7,8
0,0.061944,0.199853,0.000000,0.058541,0.060171,0.048794,0.072264,0.527604,0.065293
1,0.024428,0.580583,0.096956,0.003134,0.159963,0.089272,0.338144,1.000000,0.003519
